In [0]:

#importing all necessary libraries
import pandas as pd
import numpy as np
from sys import getsizeof

In [0]:
df_train = pd.read_csv("https://mydmtbucket.s3-ap-southeast-1.amazonaws.com/training_set_VU_DM.csv")


In [0]:
df_train.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 54 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int64  
 1   date_time                    object 
 2   site_id                      int64  
 3   visitor_location_country_id  int64  
 4   visitor_hist_starrating      float64
 5   visitor_hist_adr_usd         float64
 6   prop_country_id              int64  
 7   prop_id                      int64  
 8   prop_starrating              int64  
 9   prop_review_score            float64
 10  prop_brand_bool              int64  
 11  prop_location_score1         float64
 12  prop_location_score2         float64
 13  prop_log_historical_price    float64
 14  position                     int64  
 15  price_usd                    float64
 16  promotion_flag               int64  
 17  srch_destination_id          int64  
 18  srch_length_of_stay          int64  
 19  

In [0]:
for dtype in ['float','int','object']:
    selected_dtype = df_train.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))


Average memory usage for float columns: 36.75 MB
Average memory usage for int columns: 35.94 MB
Average memory usage for object columns: 179.69 MB


In [0]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)





In [0]:
df_train_int =  df_train.select_dtypes(include=['int'])
converted_int = df_train_int.apply(pd.to_numeric, downcast='unsigned')
print(mem_usage(df_train_int))
print(mem_usage(converted_int))
compare_ints = pd.concat([df_train_int.dtypes, converted_int.dtypes], axis=1)
compare_ints.columns =  ['before','after']
compare_ints.apply(pd.Series.value_counts)

718.75 MB
127.67 MB


,before,after
uint8,NaN,15.0
uint16,NaN,2.0
uint32,NaN,2.0
int64,19.0,NaN


In [0]:
df_train_float =  df_train.select_dtypes(include=['float'])
converted_float = df_train_int.apply(pd.to_numeric, downcast='float')
print(mem_usage(df_train_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([df_train_float.dtypes, converted_float.dtypes], axis=1)
compare_floats.columns =  ['before','after']
compare_floats.apply(pd.Series.value_counts)

1286.19 MB
359.38 MB


,before,after
float32,NaN,19.0
float64,34.0,NaN


In [0]:
optimized_df_train = df_train.copy()
optimized_df_train[converted_int.columns] = converted_int
optimized_df_train[converted_float.columns] = converted_float
print(mem_usage(df_train))
print(mem_usage(optimized_df_train))

2364.32 MB
2004.95 MB


In [0]:
date = optimized_df_train.date_time
print(mem_usage(date))
date.head()

359.38 MB


0    2013-04-04 08:32:15
1    2013-04-04 08:32:15
2    2013-04-04 08:32:15
3    2013-04-04 08:32:15
4    2013-04-04 08:32:15
Name: date_time, dtype: object

In [0]:
optimized_df_train['date_time'] = pd.to_datetime(date, format='%Y-%m-%d %H:%M:%S')
print(mem_usage(optimized_df_train))
optimized_df_train.date_time.head()

1683.40 MB


0   2013-04-04 08:32:15
1   2013-04-04 08:32:15
2   2013-04-04 08:32:15
3   2013-04-04 08:32:15
4   2013-04-04 08:32:15
Name: date_time, dtype: datetime64[ns]

In [0]:
dtypes = optimized_df_train.drop('date_time', axis=1).dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col,dtypes_type))
preview = first2pairs = {key:value for key,value in list(column_types.items())[:54]}
import pprint as pp
pp = pp.PrettyPrinter(indent=4)
pp.pprint(preview)

{   'booking_bool': 'float32',
    'click_bool': 'float32',
    'comp1_inv': 'float64',
    'comp1_rate': 'float64',
    'comp1_rate_percent_diff': 'float64',
    'comp2_inv': 'float64',
    'comp2_rate': 'float64',
    'comp2_rate_percent_diff': 'float64',
    'comp3_inv': 'float64',
    'comp3_rate': 'float64',
    'comp3_rate_percent_diff': 'float64',
    'comp4_inv': 'float64',
    'comp4_rate': 'float64',
    'comp4_rate_percent_diff': 'float64',
    'comp5_inv': 'float64',
    'comp5_rate': 'float64',
    'comp5_rate_percent_diff': 'float64',
    'comp6_inv': 'float64',
    'comp6_rate': 'float64',
    'comp6_rate_percent_diff': 'float64',
    'comp7_inv': 'float64',
    'comp7_rate': 'float64',
    'comp7_rate_percent_diff': 'float64',
    'comp8_inv': 'float64',
    'comp8_rate': 'float64',
    'comp8_rate_percent_diff': 'float64',
    'gross_bookings_usd': 'float64',
    'orig_destination_distance': 'float64',
    'position': 'float32',
    'price_usd': 'float64',
    'promoti

In [0]:
read_and_optimized = pd.read_csv('https://mydmtbucket.s3-ap-southeast-1.amazonaws.com/training_set_VU_DM.csv', dtype=column_types, parse_dates=['date_time'],infer_datetime_format=True)
print(mem_usage(read_and_optimized))
#read_and_optimized.head()

1683.40 MB


In [0]:
#pd.set_option("max_rows", None)
read_and_optimized.describe()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
count,4.958347e+06,4.958347e+06,4.958347e+06,251866.000000,252988.000000,4.958347e+06,4.958347e+06,4.958347e+06,4.950983e+06,4.958347e+06,4.958347e+06,3.867999e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,317406.000000,3.350565e+06,4.958347e+06,119930.000000,129559.000000,94439.000000,2.024672e+06,2.130269e+06,556238.000000,1.534288e+06,1.650990e+06,472797.000000,307378.000000,343663.000000,1.310860e+05,2.222373e+06,2.360020e+06,841099.000000,240157.000000,260976.000000,96174.000000,315348.000000,356422.000000,138515.000000,1.916654e+06,1.987503e+06,614730.000000,4.958347e+06,138390.000000,4.958347e+06
mean,1.663566e+05,9.634107e+00,1.708164e+02,3.374334,176.022659,1.691926e+02,6.992119e+04,3.180526e+00,3.777777e+00,6.346994e-01,2.872589e+00,1.303852e-01,4.317913e+00,1.681677e+01,2.542096e+02,2.156198e-01,1.403779e+04,2.385427e+00,3.714274e+01,1.972637e+00,3.504918e-01,1.110525e+00,5.022128e-01,-24.146418,1.301234e+03,2.959004e-01,0.479788,0.031059,244.229916,4.418987e-02,3.931006e-02,18.490732,1.729010e-02,5.776897e-02,27.071559,-0.016543,0.096653,1.753165e+02,2.267621e-02,5.142880e-02,29.851903,0.128329,0.075957,17.250473,0.145969,0.083202,19.433267,-6.089936e-02,9.962752e-03,22.430384,4.474858e-02,386.283316,2.791051e-02
std,9.594463e+04,7.577309e+00,6.544859e+01,0.692519,107.254493,6.734032e+01,4.053902e+04,1.048321e+00,1.050329e+00,4.723713e-01,1.531011e+00,1.594634e-01,1.834869e+00,1.038426e+01,1.600124e+04,4.135197e-01,8.095312e+03,2.048263e+00,5.175193e+01,8.597246e-01,7.207729e-01,4.092268e-01,4.999987e-01,15.743238,2.023951e+03,4.494789e-01,0.641565,0.229688,1165.448634,4.484236e-01,2.342388e-01,240.113851,4.575808e-01,2.948202e-01,1012.919911,0.578718,0.337165,5.757740e+03,5.217575e-01,2.678363e-01,1340.262382,0.559841,0.302668,31.160313,0.578202,0.316722,54.370221,4.691723e-01,2.029142e-01,895.965854,2.039659e-01,821.190577,1.627631e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.410000,0.000000,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-326.567500,1.000000e-02,0.000000e+00,-1.000000,-1.000000,2.000000,-1.000000e+00,-1.000000e+00,2.000000,-1.000000e+00,-1.000000e+00,2.000000,-1.000000,-1.000000,2.000000e+00,-1.000000e+00,-1.000000e+00,2.000000,-1.000000,-1.000000,2.000000,-1.000000,-1.000000,2.000000,-1.000000e+00,-1.000000e+00,2.000000,0.000000e+00,0.000000,0.000000e+00
25%,8.293600e+04,5.000000e+00,1.000000e+02,2.920000,109.810000,1.000000e+02,3.501000e+04,3.000000e+00,3.500000e+00,0.000000e+00,1.790000e+00,1.900000e-02,4.450000e+00,8.000000e+00,8.500000e+01,0.000000e+00,7.101000e+03,1.000000e+00,4.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-30.774775,1.398000e+02,0.000000e+00,0.000000,0.000000,7.000000,0.000000e+00,0.000000e+00,7.000000,0.000000e+00,0.000000e+00,7.000000,0.000000,0.000000,7.000000e+00,0.000000e+00,0.000000e+00,7.000000,0.000000,0.0

In [0]:
#test_file = pd.read_csv("https://mydmtbucket.s3-ap-southeast-1.amazonaws.com/test_set_VU_DM.csv")


In [0]:
#test_file.info()
#read_and_optimized.info()

In [0]:
#submit_data = pd.read_csv("https://mydmtbucket.s3-ap-southeast-1.amazonaws.com/submission_sample.csv")

In [0]:
#submit_data.info()